<a href="https://colab.research.google.com/github/DLPY/Unsupervised-Learning-Session-1/blob/main/K_Means_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clustering Customers based on Bank Account Data**

Detail on Data: https://www.kaggle.com/shrutimechlearn/churn-modelling

## **TODO: Download source data from Github**
!wget https://github.com/DLPY/Classification_Session_1/blob/815d80d7c1367925bc148cf698738537d7bdc1c0/Churn_Modelling.csv

In [ ]:
!pip install --upgrade kneed
!pip install colorama
!wget https://raw.githubusercontent.com/DLPY/Classification_Session_1/main/Churn_Modelling.csv

### **1. Import necessary packages for performing K-Means Clustering**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from kneed import KneeLocator
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin, silhouette_score
import seaborn as sns
from sklearn.preprocessing import StandardScaler,normalize, MinMaxScaler
from yellowbrick.cluster import SilhouetteVisualizer
from termcolor import colored
import missingno as msno 
import colorama
from colorama import Fore, Style  # maakes strings colored
from termcolor import colored, cprint

%matplotlib inline

# pd.set_option('display.max_colwidth', None)

### **2. Read data from csv file into Pandas dataframe**

In [ ]:
df = pd.read_csv('Churn_Modelling.csv')

In [ ]:
df.head(5)

In [ ]:
def first_looking(df):
    print(colored("Shape:", attrs=['bold']), df.shape,'\n',
          f"There is ", df.shape[0], " observation and ", df.shape[1], " columns in the dataset.", '\n',
          colored('-'*79, 'red', attrs=['bold']),
          colored("\nInfo:\n", attrs=['bold']), sep='')
    print(df.info(), '\n', 
          colored('-'*79, 'red', attrs=['bold']), sep='')
    print(colored("Number of Uniques:\n", attrs=['bold']), df.nunique(),'\n',
          colored('-'*79, 'red', attrs=['bold']), sep='')
    print(colored("All Columns:", attrs=['bold']), list(df.columns),'\n', 
          colored('-'*79, 'red', attrs=['bold']), sep='')

    df.columns= df.columns.str.lower().str.replace('&', '_').str.replace(' ', '_')

    print(colored("Columns after rename:", attrs=['bold']), list(df.columns),'\n',
              colored('-'*79, 'red', attrs=['bold']), sep='')
###############################################################################

In [ ]:
first_looking(df)

### **3. Clustering of Customers based on Estimated Salary and Balance in the Bank**

In [ ]:
#features = df[['creditscore', 'countrycode', 'gendercode',  'age', 'tenure', 'balance', 'numofproducts', 'hascrcard', 'isactivemember', 'estimatedsalary']]
features = df[['balance', 'estimatedsalary']]

In [ ]:
# scale features that have a wide range of data points
#scaler = MinMaxScaler()
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features[['balance', 'estimatedsalary']])

In [ ]:
# Choosing the Appropriate Number of Clusters
kmeans_kwargs = {
    'init': 'k-means++',
    'n_init': 10,
    'max_iter': 200,
    'random_state': 42,
}

# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)

In [ ]:
plt.style.use('fivethirtyeight')
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('SSE')
plt.show()

In [ ]:
# use kneed to programatically determine the number of clusters
kl = KneeLocator(
    range(1, 11), sse, curve='convex', direction='decreasing'
)

print('Optimal value for K using this method is {}'.format(kl.elbow))

In [ ]:
fig, ax = plt.subplots(5,2, figsize=(10,15))
fig.tight_layout()
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)

for i in range (2,12):
    '''
    Create KMeans instance for different number of clusters
    '''
    km = KMeans(n_clusters=i)
    q, mod = divmod(i, 2)

    labels = km.fit(scaled_features).labels_
    score_i = round(metrics.silhouette_score(scaled_features, labels, metric='euclidean'), 2)

    '''
    Create SilhouetteVisualizer instance with KMeans instance
    Fit the visualizer
    '''
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(scaled_features)
    visualizer.ax.set_title(str(i)  + " clusters - Silhouette score " + str(score_i))

In [ ]:
# Note that the optimal value for K using this method is 4.
# Ultimately, the decision on the number of clusters to use should be
#   guided by a combination of domain knowledge and clustering evaluation metrics.

In [ ]:
kmeans = KMeans(
    init='k-means++',
    n_clusters=4,
    n_init=10,
    max_iter=300,
    random_state=42
)

In [ ]:
kmeans.fit(scaled_features)

In [ ]:
# The lowest SSE value. A good model is one with low inertia AND a low number of clusters ( K ).
kmeans.inertia_

In [ ]:
# Final locations of the centroid
kmeans.cluster_centers_

In [ ]:
# The number of iterations required to converge
kmeans.n_iter_

In [ ]:
# review a sample of the labels from the overall series of labels
kmeans.labels_[:5]

In [ ]:
features['labels'] = kmeans.labels_

In [ ]:
from sklearn.decomposition import PCA
 
#Load Data
pca = PCA(2) # 2 principal components are necessary.
 
#Transform the data
df_2d = pca.fit_transform(features)
 
df_2d.shape

In [ ]:
# PCA visualisation
plt.style.use('seaborn-whitegrid')
plt.figure(figsize=(10, 6))
c_map = plt.cm.get_cmap('jet', 10)
plt.scatter(df_2d[:, 0], df_2d[:, 1], s=15, cmap=c_map, c=df['exited'])
plt.colorbar()
plt.xlabel('PC-1') , plt.ylabel('PC-2')
plt.show()

In [ ]:
#Initialize the class object
kmeans = KMeans(n_clusters=4)

#predict the labels of clusters.
label = kmeans.fit_predict(df_2d)

print(label)

In [ ]:
import numpy as np

#Getting unique labels
 
u_labels = np.unique(label)
 
#plotting the results:
 
for i in u_labels:
    plt.scatter(df_2d[label == i, 0], df_2d[label == i, 1], label=i)
plt.legend()
plt.show()

In [ ]:
#Getting the Centroids
centroids = kmeans.cluster_centers_
u_labels = np.unique(label)
 
#plotting the results:
 
for i in u_labels:
    plt.scatter(df_2d[label == i, 0], df_2d[label == i, 1], label=i)
plt.scatter(centroids[:, 0], centroids[:, 1], s=80, color='k')
plt.legend()
plt.show()